In [4]:
!pip install sentencepiece 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00 0:00:01


In [49]:
import transformers
from transformers import AutoTokenizer 
import datasets 
from loguru import logger
import tqdm
import torch

In [50]:

tokenizer = AutoTokenizer.from_pretrained("Konstantinos/el_llama_3bn")


data = datasets.load_dataset("allenai/c4", "el", split="train", streaming=True)


tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/483k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

In [52]:
tokenizer.encode("[INST]"), tokenizer.encode("[/INST]"), tokenizer.encode("<<SYS>>"), tokenizer.encode("<</SYS>>") 

([1, 4], [1, 5], [1, 6], [1, 7])

In [53]:
seed_for_shuffle = 42 
    
logger.info(f"Shuffling data with seed {seed_for_shuffle}")
data: datasets.Dataset = data.shuffle(seed=seed_for_shuffle)


2024-04-01 21:42:07.781 | INFO     | __main__:<module>:3 - Shuffling data with seed 42


In [54]:
tokenizer.encode('this [INST]')


[1, 1145, 21884, 1198, 4]

In [58]:
tokenizer.encode("hey  [INST]", add_special_tokens=True)

tokenizer.decode(4)

'[INST]'

In [62]:
all_tokens = []

tokens_so_far = 0 
docs = 0 
with tqdm.tqdm() as pbar: 
    for idx, example in enumerate(data):
        text = example['text']
        tokens = tokenizer.encode(text + '</s>')
        
        tokens_so_far += len(tokens)
        all_tokens.extend(tokens)
        if idx % 1000 == 0 and idx!=0: 
            pbar.set_description( f"Docs: {idx}, tokens seen {tokens_so_far}")
            docs+=1000
            pbar.update(docs)

Docs: 314000, tokens seen 332927530: : 49455000it [18:46, 90897.33it/s]

In [13]:
torch.save(all_tokens, "./data/training.pt")

817